## 001 - Read and filter GPS data
#### Reading and filtering large-scale GPS trajectory data (crowsourcing navigation app data)

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import csv
from datetime import datetime

startTime = datetime.now()  # Record the start time of the task

# Connect to database 
engine = create_engine('postgresql://[USERNAME]:[PASSWORD]@[HOST:PORT]/[DATABASE]') #Change credentials

# Query data from database by extracting points within a boundary for a time period in a day
# Each table in the database represent 24 hours recorded data 

df = pd.read_sql_query( """SELECT 
                                 local_time, 
                                 ST_X(points.geom) as x, 
                                 ST_Y(points.geom) as y ,
                                 speed, 
                                 session_id
                              FROM sygic_filt_05_06 as points, melb_urban_area_buffer as bndry
                                WHERE 
                                    bndry.geom && points.geom AND 
                                    ST_Contains(bndry.geom,points.geom) 
                                    AND local_time > '2016-05-06 00:00:00'
                                    AND local_time < '2016-05-06 23:59:59'
                                    ;
                        """,con=engine)

out_file = 'C:/input_files/Fri06_all_day.csv'

print (datetime.now() - startTime)
print (df.shape)
df.dtypes

0:06:30.982000
(1670159, 5)


local_time    datetime64[ns]
x                    float64
y                    float64
speed                float64
session_id            object
dtype: object

In [3]:
# Group by session_id/user and then sort by time
group = df.groupby("session_id")
sort_function = lambda x: x.sort_values('local_time', ascending = True)
df_sorted = group.apply(sort_function)
df_sorted.rename(columns = {'session_id': 'session_id_traj'}, inplace = True)

In [4]:
df_sorted.head()

local_time              x  \
session_id                                                                    
000636ed5bd044809c7fbe5fe7732a14 1299979 2016-05-06 16:36:51  320064.998538   
                                 526209  2016-05-06 16:51:40  308591.651091   
                                 526212  2016-05-06 16:51:47  308608.852370   
                                 526213  2016-05-06 16:51:48  308610.197451   
                                 396384  2016-05-06 16:53:07  308469.056067   
00083ba5cf9e4342acc2c9b0e801023b 46385   2016-05-06 08:30:53  326866.339471   
                                 46386   2016-05-06 08:30:57  326800.156150   
                                 46387   2016-05-06 08:31:02  326727.227714   
                                 46388   2016-05-06 08:31:07  326690.785614   
                                 28288   2016-05-06 08:31:12  326676.048057   
                                 28289   2016-05-06 08:31:17  326621.981994   
                                 28290   2016-05-06 08:31:22  326550.505131   
                                 28291   2016-05-06 08:31:26  326485.790873   
                                 28292   2016-05-06 08:31:32  326399.469439   
                                 30035   2016-05-06 08:31:37  326336.046952   
                                 30036   2016-05-06 08:31:42  326291.318034   
                                 30037   2016-05-06 08:31:47  326268.668996   
                                 30038   2016-05-06 08:31:53  326248.203005   
                                 883871  2016-05-06 08:31:57  326239.168413   
                                 883872  2016-05-06 08:32:02  326242.884230   
                                 883873  2016-05-06 08:32:07  326273.687648   
                                 883874  2016-05-06 08:32:12  326307.182066   
                                 883875  2016-05-06 08:32:17  326320.592310   
                                 883876  2016-05-06 08:32:22  326323.167545   
                                 883877  2016-05-06 08:32:27  326325.813992   
                                 883878  2016-05-06 08:32:32  326327.106062   
                                 883879  2016-05-06 08:32:35  326328.451540   
                                 883880  2016-05-06 08:37:46  326345.229174   
                                 883881  2016-05-06 08:37:50  326347.644217   
                                 883882  2016-05-06 08:41:12  326346.298743   
...                                                      ...            ...   
fff5f0da8a254a99a840b6b386a67333 583347  2016-05-06 10:42:54  303981.651164   
                                 393838  2016-05-06 10:42:59  303979.011997   
                                 583348  2016-05-06 10:43:04  303976.453342   
                                 583349  2016-05-06 10:43:42  303973.532389   
                                 583350  2016-05-06 10:43:47  303973.532389   
                                 583351  2016-05-06 10:43:51  303973.532389   
                                 838460  2016-05-06 10:43:53  303973.502198   
                                 838461  2016-05-06 10:44:33  303973.532389   
                                 583352  2016-05-06 10:44:39  303973.532389   
                                 838462  2016-05-06 10:44:44  303972.288286   
                                 838463  2016-05-06 10:44:49  303973.713540   
                                 1107548 2016-05-06 10:44:54  303973.713540   
                                 583353  2016-05-06 10:44:59  303977.717573   
                                 1107549 2016-05-06 10:45:04  303979.062315   
                                 1107550 2016-05-06 10:45:05  303979.062315   
                                 583354  2016-05-06 10:45:29  303973.421686   
                                 393839  2016-05-06 10:45:31  303976.141365   
                                 838469  2016-05-06 10:46:04  303985.765898   
                     

In [5]:
#Calculate distance, delta time and speed for each record grouped by session id 

#Differences in X and Y coordinates and in time
dx = (df_sorted['x'] - df_sorted.groupby('session_id_traj')['x'].shift())
dy = (df_sorted['y'] - df_sorted.groupby('session_id_traj')['y'].shift())
dt = (df_sorted['local_time'] - df_sorted.groupby('session_id_traj')['local_time'].shift())

df_sorted['dist'] = (np.sqrt(dx**2 + dy**2))/1000 # distance in km 
df_sorted['time_dif'] = dt.astype('timedelta64[s]') # time difference in seconds
df_sorted['speed_calc'] = df_sorted['dist']/(df_sorted['time_dif']/3600)  #speed in km/hr

# Remove potential outliers - Speed higher than 140 km/hrs and time differences larger than 15 seconds
# (GPS data is recorded every 5 seconds, more than 15 seconds can be related to potential signal issues)
df_sorted.loc[df_sorted['speed_calc'] > 140, 'speed_calc'] = np.nan
df_sorted.loc[(df_sorted['time_dif'] > 15), 'speed_calc'] = np.nan

In [6]:
df_sorted

local_time              x  \
session_id                                                                    
000636ed5bd044809c7fbe5fe7732a14 1299979 2016-05-06 16:36:51  320064.998538   
                                 526209  2016-05-06 16:51:40  308591.651091   
                                 526212  2016-05-06 16:51:47  308608.852370   
                                 526213  2016-05-06 16:51:48  308610.197451   
                                 396384  2016-05-06 16:53:07  308469.056067   
00083ba5cf9e4342acc2c9b0e801023b 46385   2016-05-06 08:30:53  326866.339471   
                                 46386   2016-05-06 08:30:57  326800.156150   
                                 46387   2016-05-06 08:31:02  326727.227714   
                                 46388   2016-05-06 08:31:07  326690.785614   
                                 28288   2016-05-06 08:31:12  326676.048057   
                                 28289   2016-05-06 08:31:17  326621.981994   
                                 28290   2016-05-06 08:31:22  326550.505131   
                                 28291   2016-05-06 08:31:26  326485.790873   
                                 28292   2016-05-06 08:31:32  326399.469439   
                                 30035   2016-05-06 08:31:37  326336.046952   
                                 30036   2016-05-06 08:31:42  326291.318034   
                                 30037   2016-05-06 08:31:47  326268.668996   
                                 30038   2016-05-06 08:31:53  326248.203005   
                                 883871  2016-05-06 08:31:57  326239.168413   
                                 883872  2016-05-06 08:32:02  326242.884230   
                                 883873  2016-05-06 08:32:07  326273.687648   
                                 883874  2016-05-06 08:32:12  326307.182066   
                                 883875  2016-05-06 08:32:17  326320.592310   
                                 883876  2016-05-06 08:32:22  326323.167545   
                                 883877  2016-05-06 08:32:27  326325.813992   
                                 883878  2016-05-06 08:32:32  326327.106062   
                                 883879  2016-05-06 08:32:35  326328.451540   
                                 883880  2016-05-06 08:37:46  326345.229174   
                                 883881  2016-05-06 08:37:50  326347.644217   
                                 883882  2016-05-06 08:41:12  326346.298743   
...                                                      ...            ...   
fff5f0da8a254a99a840b6b386a67333 583347  2016-05-06 10:42:54  303981.651164   
                                 393838  2016-05-06 10:42:59  303979.011997   
                                 583348  2016-05-06 10:43:04  303976.453342   
                                 583349  2016-05-06 10:43:42  303973.532389   
                                 583350  2016-05-06 10:43:47  303973.532389   
                                 583351  2016-05-06 10:43:51  303973.532389   
                                 838460  2016-05-06 10:43:53  303973.502198   
                                 838461  2016-05-06 10:44:33  303973.532389   
                                 583352  2016-05-06 10:44:39  303973.532389   
                                 838462  2016-05-06 10:44:44  303972.288286   
                                 838463  2016-05-06 10:44:49  303973.713540   
                                 1107548 2016-05-06 10:44:54  303973.713540   
                                 583353  2016-05-06 10:44:59  303977.717573   
                                 1107549 2016-05-06 10:45:04  303979.062315   
                                 1107550 2016-05-06 10:45:05  303979.062315   
                                 583354  2016-05-06 10:45:29  303973.421686   
                                 393839  2016-05-06 10:45:31  303976.141365   
                                 838469  2016-05-06 10:46:04  303985.765898   
                     

In [7]:
# Evaluate summary statistics each trajectory/session id by their speed values
traj = df_sorted.groupby("session_id_traj")
summary_stats = traj['speed_calc'].describe()
summary_stats

,count,mean,std,min,25%,50%,75%,max
session_id_traj,,,,,,,,
000636ed5bd044809c7fbe5fe7732a14,2.0,7.917101,4.346596,4.843593,6.380347,7.917101,9.453854,10.990608
00083ba5cf9e4342acc2c9b0e801023b,108.0,16.136319,20.476890,0.000000,1.476228,4.702159,25.050121,72.290475
0008e8b8946f4f7993d1f8b6a71a8ed3,40.0,33.456715,32.442346,0.000000,4.020589,17.689619,64.254447,87.946281
000b7025eab147e995a1ee4f02416a29,11.0,0.802880,1.038931,0.000000,0.000000,0.000000,1.390644,2.670917
000cac229635480aa97d63b5d4b2b742,11.0,1.162645,2.349878,0.000000,0.000000,0.000000,1.507083,7.841626
000dd5ee67464bb3b3edc74c82019304,154.0,84.549088,20.535010,20.327614,82.263502,92.452469,97.567253,119.490258
00101a3fe7b54896ac1562f78e36d3b6,7.0,4.568854,5.547332,0.000000,1.737512,2.664623,4.656363,16.529604
0010be14d6f6477d8a0af3232f478999,174.0,17.058623,24.432119,0.000000,0.000000,1.569437,32.642768,85.163656
0010f859131e437fa347f2fd7a8b2b10,1.0,6.010021,NaN,6.010021,6.010021,6.010021,6.010021,6.010021


In [8]:
# Remove potential outliers by evaluating mean speed values
speed_quantile = summary_stats['mean'].quantile(.25)
print summary_stats['count'].describe()
print summary_stats['mean'].describe()

count    23507.000000
mean        67.898286
std        274.853147
min          0.000000
25%          2.000000
50%         11.000000
75%         40.000000
max      13149.000000
Name: count, dtype: float64
count    20828.000000
mean        14.843655
std         19.438483
min          0.000000
25%          1.292453
50%          5.892376
75%         22.190030
max        118.714108
Name: mean, dtype: float64


In [9]:
# Filter noise data that shows no movement (speed variable)

# Drop sessions where there are no values for speed calculations
keep_sessions = (summary_stats.dropna(subset = ['max','std','min'])).index.values.tolist()
print len(keep_sessions)
df_sorted = df_sorted[df_sorted['session_id_traj'].isin(keep_sessions)]
#Filter by speeed values (no movement)  
keep_sessions = (summary_stats.loc[(summary_stats['mean'] >= speed_quantile)]).index.values.tolist()
print len(keep_sessions)
df_sorted = df_sorted[df_sorted['session_id_traj'].isin(keep_sessions)]

18668
15621


In [10]:
# Evaluate summary statistics of each trajectory/session id by their distance values (km)

sessions_dist = df_sorted.groupby('session_id_traj')['dist'].agg(['sum','count','max','min'])
sessions_dist.sort_values('sum', ascending = True)

,sum,count,max,min
session_id_traj,,,,
8fca4ade5ea04ad8b168288decf1eb5f,0.000847,2,0.000847,0.000000
814529954ee7423581ae3a02c03d48ee,0.000847,2,0.000847,0.000000
f17f0d5f4df04892a104e2d72f8a117e,0.001270,3,0.001270,0.000000
d138efea7ee840ceb72f32df2b1df49f,0.001270,2,0.001270,0.000000
f85a0e76683f4306b3ee373772fb912c,0.001343,2,0.001343,0.000000
f9b56934607a4137b4f4d7893c0085e2,0.001343,2,0.001343,0.000000
385e11e0dd3b47dd8f171ec7a8ffd8ca,0.001344,2,0.001344,0.000000
a05b83b343344d9a9e1dcc6de957e2f7,0.001845,2,0.001845,0.000000
35615e2dd8e74b96a11d84e4dba2aaef,0.002117,2,0.001270,0.000847


In [11]:
# Filter noise data that shows no movement (distance variable)
keep_sessions = (sessions_dist.loc[sessions_dist['sum'] > 0.050]).index.values.tolist()
print len(keep_sessions)
df_sorted = df_sorted[df_sorted['session_id_traj'].isin(keep_sessions)]
df_sorted.shape

11712


(1316538, 8)

In [12]:
# Get final summary statistcs for remaining trajectories/sessions 

traj_filt = df_sorted.groupby("session_id_traj")
summary_stats_filt = traj_filt['speed_calc'].describe()

print summary_stats_filt['count'].describe()
print summary_stats_filt['mean'].describe()

count    11712.000000
mean       107.933402
std        303.685216
min          2.000000
25%         11.000000
50%         24.000000
75%         77.000000
max       9044.000000
Name: count, dtype: float64
count    11712.000000
mean        22.963912
std         20.494111
min          1.293396
25%          6.703300
50%         17.223635
75%         32.886113
max        114.685371
Name: mean, dtype: float64


In [19]:
# Export list of filtered data for map maptching process. 
list_map_matched = pd.read_csv('C:/sygic/output_files/Friday06_matchable_sessions.csv')
matchable_sessions = list(list_map_matched.session_id)

In [21]:
print len(matchable_sessions)
df_match = df_sorted[df_sorted['session_id_traj'].isin(matchable_sessions)]

7239


In [23]:
df_match.shape

(677295, 8)

In [13]:
# Check if sorting is correct by picking one random session_id 

#df_test = df_sorted.loc[df_sorted['session_id_traj'] == 'b46c691d8c6e410b8fb3720b99a089c2']
#df_test['dist'].agg(['sum','count','max','min'])
#sessions_dist.sort_values('sum', ascending = True)
#summary_stats_test= df_test['dist'].describe()
#summary_stats_test

In [15]:
# Export sorted table as CSV 
df_sorted.to_csv(out_file,encoding = 'utf-8')